### Access your data in S3 through EC2

In [115]:
import pandas as pd

path = "2013-12 - Citi Bike trip data.csv"

df = pd.read_csv(path)

In [116]:
df.head()

tripduration            starttime             stoptime  start station id  \
0          1015  2013-12-01 00:00:03  2013-12-01 00:16:58               401   
1           962  2013-12-01 00:00:07  2013-12-01 00:16:09               312   
2           768  2013-12-01 00:00:15  2013-12-01 00:13:03               326   
3           218  2013-12-01 00:00:49  2013-12-01 00:04:27              2021   
4           550  2013-12-01 00:01:28  2013-12-01 00:10:38               439   

        start station name  start station latitude  start station longitude  \
0  Allen St & Rivington St               40.720196               -73.989978   
1  Allen St & E Houston St               40.722055               -73.989111   
2          E 11 St & 1 Ave               40.729538               -73.984267   
3          W 45 St & 8 Ave               40.759291               -73.988597   
4           E 4 St & 2 Ave               40.726281               -73.989780   

   end station id            end station name  end station latitude  \
0             476             E 31 St & 3 Ave             40.743943   
1             223             W 13 St & 7 Ave             40.737815   
2             291  Madison St & Montgomery St             40.713126   
3             447             8 Ave & W 52 St             40.763707   
4             296        Division St & Bowery             40.714131   

   end station longitude  bikeid    usertype birth year  gender  
0             -73.979661   14729  Subscriber       1979       2  
1             -73.999947   17871  Subscriber       1970       2  
2             -73.984844   18508  Subscriber       1982       2  
3             -73.985162   17745  Subscriber       1976       1  
4             -73.997047   16947  Subscriber       1983       1

### Rename the columns of of the dataframe
Remove the space in names would be easier for the following indexing

In [117]:
df_bike = df.rename(columns={"start station id": "start_station_id", 
                             "start station name": "start_station_name",
                             "start station latitude": "start_station_lat",
                             "start station longitude": "start_station_lon", 
                             "end station id": "end_station_id", 
                             "end station name": "end_station_name", 
                             "end station latitude": "end_station_lat",
                             "end station longitude": "end_station_lon", 
                             "birth year": "birth_year"})
df_bike.head()

tripduration            starttime             stoptime  start_station_id  \
0          1015  2013-12-01 00:00:03  2013-12-01 00:16:58               401   
1           962  2013-12-01 00:00:07  2013-12-01 00:16:09               312   
2           768  2013-12-01 00:00:15  2013-12-01 00:13:03               326   
3           218  2013-12-01 00:00:49  2013-12-01 00:04:27              2021   
4           550  2013-12-01 00:01:28  2013-12-01 00:10:38               439   

        start_station_name  start_station_lat  start_station_lon  \
0  Allen St & Rivington St          40.720196         -73.989978   
1  Allen St & E Houston St          40.722055         -73.989111   
2          E 11 St & 1 Ave          40.729538         -73.984267   
3          W 45 St & 8 Ave          40.759291         -73.988597   
4           E 4 St & 2 Ave          40.726281         -73.989780   

   end_station_id            end_station_name  end_station_lat  \
0             476             E 31 St & 3 Ave        40.743943   
1             223             W 13 St & 7 Ave        40.737815   
2             291  Madison St & Montgomery St        40.713126   
3             447             8 Ave & W 52 St        40.763707   
4             296        Division St & Bowery        40.714131   

   end_station_lon  bikeid    usertype birth_year  gender  
0       -73.979661   14729  Subscriber       1979       2  
1       -73.999947   17871  Subscriber       1970       2  
2       -73.984844   18508  Subscriber       1982       2  
3       -73.985162   17745  Subscriber       1976       1  
4       -73.997047   16947  Subscriber       1983       1

### Count the number of trajectories from and stop at different stations

In [118]:
start_station = df_bike.groupby('start_station_id').size().to_frame('size')
start_station

size
start_station_id      
72                1213
79                 995
82                 558
83                 726
116               2763
...                ...
2017               665
2021              2015
2022               686
2023               771
3002              1236

[329 rows x 1 columns]

In [119]:
end_station = df_bike.groupby('end_station_id').size().to_frame('size')
end_station

size
end_station_id      
72              1269
79              1024
82               582
83               669
116             2805
...              ...
2017             551
2021            1809
2022             740
2023             749
3002            1411

[329 rows x 1 columns]

#### Rename the axis and prepare for merging
The start station and the end station are both the station id, we cand change the number both as station id

In [71]:
s_station = start_station.rename_axis("station_id")
e_station = end_station.rename_axis("station_id")

#### Merge the start and end station dataframes

In [124]:
SE_station = s_station.merge(e_station, on='station_id')

# rename the size column as number of trips start and end
SE_station = SE_station.rename(columns={"size_x": 'start', "size_y": "end"})
SE_station.head()

start   end
station_id             
72           1213  1269
79            995  1024
82            558   582
83            726   669
116          2763  2805

#### Our table has no spatial information, we need join with the original data that has the cooridnate information
Change the index name back to the start_station_id, we going to use this join with the original table.

In [125]:
SE_station = SE_station.rename_axis("start_station_id")
SE_station

start   end
start_station_id             
72                 1213  1269
79                  995  1024
82                  558   582
83                  726   669
116                2763  2805
...                 ...   ...
2017                665   551
2021               2015  1809
2022                686   740
2023                771   749
3002               1236  1411

[329 rows x 2 columns]

#### Extract the lon, lat of each stations by removing all duplicate stations info, and then merge with the created table

In [126]:
df_bike_coord = df_bike.drop_duplicates(subset = ["start_station_id"])

df_bike_coord = df_bike_coord.merge(SE_station, on="start_station_id")
df_bike_coord

tripduration            starttime             stoptime  start_station_id  \
0            1015  2013-12-01 00:00:03  2013-12-01 00:16:58               401   
1             962  2013-12-01 00:00:07  2013-12-01 00:16:09               312   
2             768  2013-12-01 00:00:15  2013-12-01 00:13:03               326   
3             218  2013-12-01 00:00:49  2013-12-01 00:04:27              2021   
4             550  2013-12-01 00:01:28  2013-12-01 00:10:38               439   
..            ...                  ...                  ...               ...   
324          1333  2013-12-01 13:46:57  2013-12-01 14:09:10               419   
325           177  2013-12-01 14:02:18  2013-12-01 14:05:15               443   
326           170  2013-12-01 14:58:11  2013-12-01 15:01:01               298   
327           934  2013-12-01 15:02:02  2013-12-01 15:17:36              2005   
328           375  2013-12-02 00:52:13  2013-12-02 00:58:28               409   

           start_station_name  start_station_lat  start_station_lon  \
0     Allen St & Rivington St          40.720196         -73.989978   
1     Allen St & E Houston St          40.722055         -73.989111   
2             E 11 St & 1 Ave          40.729538         -73.984267   
3             W 45 St & 8 Ave          40.759291         -73.988597   
4              E 4 St & 2 Ave          40.726281         -73.989780   
..                        ...                ...                ...   
324    Carlton Ave & Park Ave          40.695807         -73.973556   
325    Bedford Ave & S 9th St          40.708531         -73.964090   
326   3 Ave & Schermerhorn St          40.686832         -73.979677   
327    Railroad Ave & Kay Ave          40.705312         -73.971001   
328  DeKalb Ave & Skillman St          40.690649         -73.956431   

     end_station_id            end_station_name  end_station_lat  \
0               476             E 31 St & 3 Ave        40.743943   
1               223             W 13 St & 7 Ave        40.737815   
2               291  Madison St & Montgomery St        40.713126   
3               447             8 Ave & W 52 St        40.763707   
4               296        Division St & Bowery        40.714131   
..              ...                         ...              ...   
324             293       Lafayette St & E 8 St        40.730287   
325             443      Bedford Ave & S 9th St        40.708531   
326             395   Bond St & Schermerhorn St        40.688070   
327             310         State St & Smith St        40.689269   
328             396  Lefferts Pl & Franklin Ave        40.680342   

     end_station_lon  bikeid    usertype birth_year  gender  start   end  
0         -73.979661   14729  Subscriber       1979       2   1968  1972  
1         -73.999947   17871  Subscriber       1970       2   1739  1758  
2         -73.984844   18508  Subscriber       1982       2   2241  2136  
3         -73.985162   17745  Subscriber       1976       1   2015  1809  
4         -73.997047   16947  Subscriber       1983       1   1752  1923  
..               ...     ...         ...        ...     ...    ...   ...  
324       -73.990765   15217  Subscriber       1983       1     42    36  
325       -73.964090   18525  Subscriber       1953       1     80    75  
326       -73.984106   14589  Subscriber       1960       1    173   203  
327       -73.989129   16410  Subscriber       1984       1     47    54  
328       -73.955769   15586  Subscriber       1966       1    195   201  

[329 rows x 17 columns]

#### Create a GeoPandas based on the lon, lat info in the dataframe

In [128]:
# from shapely.geometry import Point
# from geopandas import GeoDataFrame

# geometry = [Point(xy) for xy in zip(df_bike_coord.end_station_lon, df_bike_coord.end_station_lat)]
# gdf = GeoDataFrame(df_bike_coord, crs="EPSG:4326", geometry=geometry)


from geopandas import GeoDataFrame as gdf
import geopandas as gpd
import pandas as pd

crs = {'init': 'epsg:4326'} #http://www.spatialreference.org/ref/epsg/2263/

# create a geo-dataframe
points_gdf= gpd.GeoDataFrame(df_bike_coord, crs=crs, \
                geometry=gpd.points_from_xy(df_bike_coord.end_station_lon, df_bike_coord.end_station_lat))

# save the geo-dataframe as a shapefile
points_gdf.to_file("stations.shp")

### Homework:
Check the number of male and female trips for different stations